# Training an NVIDIA HugeCTR model with Vertex AI Training



In [1]:
import json
import os
import time
import nvtabular as nvt
import shutil

from nvtabular.columns.schema import ColumnSchema, Schema
from nvtabular.tags import Tags
from google.cloud import aiplatform

## Configure environment settings


In [2]:
PROJECT = 'jk-mlops-dev'
REGION = 'us-central1'
STAGING_BUCKET = 'gs://jk-vertex-merlin'
LOCAL_STAGING_PATH = '/home/jupyter/staging'
VERTEX_SA = 'vertex-sa@jk-mlops-dev.iam.gserviceaccount.com'

In [3]:
if os.path.isdir(LOCAL_STAGING_PATH):
    shutil.rmtree(LOCAL_STAGING_PATH)
os.makedirs(LOCAL_STAGING_PATH)

### Initialize Vertex AI SDK

In [4]:
aiplatform.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=STAGING_BUCKET
)

## Prepare a custom training container

In [5]:
IMAGE_NAME = 'hugectr_deepfm'
IMAGE_URI = f'gcr.io/{PROJECT}/{IMAGE_NAME}'
DOCKERFILE = 'src/training/hugectr'

In [6]:
! gcloud builds submit --tag {IMAGE_URI} {DOCKERFILE}

Creating temporary tarball archive of 13 file(s) totalling 76.1 KiB before compression.
Uploading tarball of [src/training/hugectr] to [gs://jk-mlops-dev_cloudbuild/source/1633737398.03385-3be31f73befe4032a6582b777a237e23.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/jk-mlops-dev/locations/global/builds/0cd42bcb-6f34-4cb3-b2f0-cb0c299fb107].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/0cd42bcb-6f34-4cb3-b2f0-cb0c299fb107?project=895222332033].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "0cd42bcb-6f34-4cb3-b2f0-cb0c299fb107"

FETCHSOURCE
Fetching storage object: gs://jk-mlops-dev_cloudbuild/source/1633737398.03385-3be31f73befe4032a6582b777a237e23.tgz#1633737398376135
Copying gs://jk-mlops-dev_cloudbuild/source/1633737398.03385-3be31f73befe4032a6582b777a237e23.tgz#1633737398376135...
/ [1 files][ 16.5 KiB/ 16.5 KiB]                                                
Operation completed over 1 o

## Configure a training job

### Set paths to training and validation datasets

In [21]:
DATA_ROOT = 'gs://jk-criteo-bucket/criteo_processed_parquet'
TRAIN_DATA = f'{DATA_ROOT}/train/_file_list.txt'
VALID_DATA = f'{DATA_ROOT}/valid/_file_list.txt'
TRAIN_METADATA = f'{DATA_ROOT}/train/_metadata.json'
VALID_METADATA = f'{DATA_ROOT}/valid/_metadata.json'
SCHEMA_PATH = f'{DATA_ROOT}/train/schema.pbtxt'

### Retrieve cardinalities for categorical columns

In [22]:
LOCAL_SCHEMA_PATH = f'{LOCAL_STAGING_PATH}/schema.pbtxt'

!gsutil cp {SCHEMA_PATH} {LOCAL_SCHEMA_PATH}

Copying gs://jk-criteo-bucket/criteo_processed_parquet/train/schema.pbtxt...
/ [1 files][ 20.8 KiB/ 20.8 KiB]                                                
Operation completed over 1 objects/20.8 KiB.                                     


In [23]:
schema = Schema.load_protobuf(LOCAL_SCHEMA_PATH)

In [24]:
def retrieve_cardinalities(schema):
    cardinalities = {key: value.properties['embedding_sizes']['cardinality'] 
                     for key, value in schema.column_schemas.items()
                     if Tags.CATEGORICAL in value.tags}
    
    return cardinalities
    
    
cardinalities = retrieve_cardinalities(schema)
cardinalities

{'C1': 18792578.0,
 'C2': 35176.0,
 'C3': 17091.0,
 'C4': 7383.0,
 'C5': 20154.0,
 'C6': 4.0,
 'C7': 7075.0,
 'C8': 1403.0,
 'C9': 63.0,
 'C10': 12687136.0,
 'C11': 1054830.0,
 'C12': 297377.0,
 'C13': 11.0,
 'C14': 2209.0,
 'C15': 10933.0,
 'C16': 113.0,
 'C17': 4.0,
 'C18': 972.0,
 'C19': 15.0,
 'C20': 19550853.0,
 'C21': 5602712.0,
 'C22': 16779972.0,
 'C23': 375290.0,
 'C24': 12292.0,
 'C25': 101.0,
 'C26': 35.0}

### Configure worker pools

#### Training module

In [25]:
TRAINING_MODULE = 'trainer.task'

#### HugeCTR model and trainer configuration

In [37]:
NUM_EPOCHS = 0
MAX_ITERATIONS = 5000
EVAL_INTERVAL = 500
EVAL_BATCHES = 300
DISPLAY_INTERVAL = 100
SNAPSHOT_INTERVAL = 0
WORKSPACE_SIZE_PER_GPU = 61
PER_GPU_BATCHSIZE = 2048
LR = 0.001
DROPOUT_RATE = 0.5
NUM_WORKERS = 12
SLOT_SIZE_ARRAY = json.dumps(
    [int(cardinality) for cardinality in cardinalities.values()]).replace(' ', '')

### Calculate a number of batches in a validation dataset


In [27]:
LOCAL_METADATA_PATH = f'{LOCAL_STAGING_PATH}/_metadata.json'

!gsutil cp {VALID_METADATA} {LOCAL_METADATA_PATH}

Copying gs://jk-criteo-bucket/criteo_processed_parquet/valid/_metadata.json...
/ [1 files][  3.1 KiB/  3.1 KiB]                                                
Operation completed over 1 objects/3.1 KiB.                                      


In [34]:
with open(LOCAL_METADATA_PATH, 'r') as f:
    valid_metadata = json.load(f)

num_rows = sum([file_info['num_rows'] for file_info in valid_metadata['file_stats']])
all_eval_batches = num_rows // PER_GPU_BATCHSIZE

#### Training node configuration

In [35]:
MACHINE_TYPE = 'a2-highgpu-4g'
ACCELERATOR_TYPE = 'NVIDIA_TESLA_A100'
ACCELERATOR_NUM = 4

#### Worker pool specification

In [38]:
batchsize = PER_GPU_BATCHSIZE * ACCELERATOR_NUM
gpus = json.dumps([list(range(ACCELERATOR_NUM))]).replace(' ','')
                 
worker_pool_specs =  [
    {
        "machine_spec": {
            "machine_type": MACHINE_TYPE,
            "accelerator_type": ACCELERATOR_TYPE,
            "accelerator_count": ACCELERATOR_NUM,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": IMAGE_URI,
            "command": ["python", "-m", TRAINING_MODULE],
            "args": [
                '--batchsize=' + str(batchsize),
                '--train_data=' + TRAIN_DATA.replace('gs://', '/gcs/'), 
                '--valid_data=' + VALID_DATA.replace('gs://', '/gcs/'),
                '--slot_size_array=' + SLOT_SIZE_ARRAY,
                '--max_iter=' + str(MAX_ITERATIONS),
                '--max_eval_batches=' + str(EVAL_BATCHES),
                '--eval_batches=' + str(all_eval_batches),
                '--dropout_rate=' + str(DROPOUT_RATE),
                '--num_workers=' + str(NUM_WORKERS),
                '--num_epochs=' + str(NUM_EPOCHS),
                '--eval_interval=' + str(EVAL_INTERVAL),
                '--snapshot=' + str(SNAPSHOT_INTERVAL),
                '--display_interval=' + str(DISPLAY_INTERVAL),
                '--workspace_size_per_gpu=' + str(WORKSPACE_SIZE_PER_GPU),
                '--gpus=' + gpus,
            ],
        },
    }
]

## Submit and monitor a training job

In [ ]:
job_name = 'HUGECTR_{}'.format(time.strftime("%Y%m%d_%H%M%S"))

job = aiplatform.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
)
job.run(
    sync=True,
    service_account=VERTEX_SA,
    restart_job_on_worker_restart=False
)

INFO:google.cloud.aiplatform.jobs:Creating CustomJob
INFO:google.cloud.aiplatform.jobs:CustomJob created. Resource name: projects/895222332033/locations/us-central1/customJobs/4259237566143791104
INFO:google.cloud.aiplatform.jobs:To use this CustomJob in another session:
INFO:google.cloud.aiplatform.jobs:custom_job = aiplatform.CustomJob.get('projects/895222332033/locations/us-central1/customJobs/4259237566143791104')
INFO:google.cloud.aiplatform.jobs:View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/4259237566143791104?project=895222332033


## Submit and monitor a hyperparameter tuning job